In [1]:
import os
import json
import numpy as np
import pickle
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [2]:
file = "fua.gpkg"
fua =geopandas.read_file(os.path.join("data", "ghsl",file))

In [3]:
fua

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry
0,1.0,1.0,5959,Herat,1.0,AFG,Afghanistan,139.0,60.0,888811.175807,844574.564331,44236.611476,"MULTIPOLYGON (((5529000.000 4155000.000, 55310..."
1,192.0,1.0,5964,Guzarah,1.0,AFG,Afghanistan,32.0,16.0,169489.573231,160204.260864,9285.312366,"MULTIPOLYGON (((5541000.000 4139000.000, 55420..."
2,354.0,1.0,5968,Shindand,0.0,AFG,Afghanistan,12.0,12.0,147553.403870,147553.403870,0.000000,"MULTIPOLYGON (((5573000.000 4030000.000, 55740..."
3,505.0,1.0,5970,Qala i Naw,0.0,AFG,Afghanistan,3.0,3.0,79809.722656,79809.722656,0.000000,"MULTIPOLYGON (((5592000.000 4221000.000, 55900..."
4,648.0,1.0,5973,Farah,1.0,AFG,Afghanistan,32.0,16.0,131508.797060,122843.460327,8665.336733,"MULTIPOLYGON (((5607000.000 3923000.000, 56080..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9026,1739.0,1.0,4191,Triangle,0.0,ZWE,Zimbabwe,5.0,5.0,61859.251465,61859.251465,0.000000,"MULTIPOLYGON (((3023000.000 -2577000.000, 3021..."
9027,1908.0,1.0,4206,Glenclova,0.0,ZWE,Zimbabwe,4.0,4.0,158476.038086,158476.038086,0.000000,"MULTIPOLYGON (((3033000.000 -2449000.000, 3032..."
9028,2062.0,1.0,4222,Bindura,1.0,ZWE,Zimbabwe,54.0,16.0,109737.644325,74992.162598,34745.481728,"MULTIPOLYGON (((3053000.000 -2126000.000, 3054..."
9029,2137.0,1.0,4236,Marondera,1.0,ZWE,Zimbabwe,23.0,12.0,61332.544647,52538.743042,8793.801605,"MULTIPOLYGON (((3067000.000 -2235000.000, 3065..."


In [13]:
egypt_fua = fua[fua.Cntry_name=="Egypt"]
turkey_fua = fua[fua.Cntry_name=="Turkey"]
south_africa_fua = fua[fua.Cntry_name=="SouthAfrica"]

In [19]:
def get_fua(fua, city):
    '''Returns the corresponding FUA given a city name

    Args:
        fua (geopandas dataframe): GHSL dataset
        city (string): City name with capital first letter, eg. "Paris"
    '''
    result = fua[fua.eFUA_name==city]
    if result is None:
        print("FUA not found")
    return result

In [55]:
cairo_fua = get_fua(fua, "Adana")
print(cairo_fua.geometry.bounds.to_numpy())

[[3059000. 4443000. 3092000. 4459000.]]


In [61]:
def write_json_fua(filename, fua, cities):
    '''Populate a JSON file with the bounding boxes of cities FUA's

    Args:
        filename (string): json filename
        fua (geopandas dataframe): fua dataframe
        cities (list): list of cities names
    '''
    with open( filename, "r" ) as jsonFile:
        data = json.load(jsonFile)
    
    for city in cities:
        city_fua = get_fua(fua, city)
        if city_fua is not None:
            bounds = city_fua.geometry.bounds.to_numpy()[0] / 100000
            box = [
                [bounds[3], bounds[0]],
                [bounds[1], bounds[2]]
            ]
            data[city]["box"] = box
    with open( filename, "w" ) as jsonFile:
        json.dump(data, jsonFile)

In [62]:
cities = [
    "Cairo",
    "Johannesburg",
    "Adana",
    "Ordu",
    "Trabzon",
    "Manisa"
]

In [63]:
write_json_fua("bounding_box.json", fua, cities)